In [33]:
import tensorflow as tf
import os 
import tensorflow.keras as keras
import re
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [20]:
IMAGE_SHAPE = (224,224)
numEPOCHS = 3
BATCH_SIZE = 64 

In [21]:
TRAIN_DIR = "cropdisease/train"
VALID_DIR = "cropdisease/valid/"

In [22]:
#os.listdir(TRAIN_DIR)
len(os.listdir(TRAIN_DIR)),len(os.listdir(VALID_DIR))

(20, 20)

In [23]:
CLASS_NUMS = len(os.listdir(TRAIN_DIR))
CLASS_NUMS

20

In [24]:
training_aug = ImageDataGenerator(rescale=1./255,
                                  rotation_range=45,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

In [25]:
testing_aug = ImageDataGenerator(rescale = 1/255.0)

In [26]:
training_data = training_aug.flow_from_directory(TRAIN_DIR,
                                   batch_size=BATCH_SIZE,
                                   target_size=IMAGE_SHAPE,
                                   shuffle=True,
                                   class_mode='categorical')

Found 36750 images belonging to 20 classes.


In [27]:
testing_data = testing_aug.flow_from_directory(VALID_DIR,
                                   batch_size=BATCH_SIZE,
                                   target_size=IMAGE_SHAPE,
                                   class_mode='categorical')

Found 9148 images belonging to 20 classes.


In [31]:
reversed_dict = {value: key for key, value in training_data.class_indices.items()}
reversed_dict

{0: 'Apple___Cedar_apple_rust',
 1: 'Apple___healthy',
 2: 'Blueberry___healthy',
 3: 'Cherry_(including_sour)___Powdery_mildew',
 4: 'Cherry_(including_sour)___healthy',
 5: 'Corn_(maize)___Common_rust_',
 6: 'Corn_(maize)___healthy',
 7: 'Grape___Black_rot',
 8: 'Grape___healthy',
 9: 'Peach___Bacterial_spot',
 10: 'Peach___healthy',
 11: 'Pepper,_bell___Bacterial_spot',
 12: 'Pepper,_bell___healthy',
 13: 'Potato___Early_blight',
 14: 'Potato___Late_blight',
 15: 'Potato___healthy',
 16: 'Strawberry___Leaf_scorch',
 17: 'Strawberry___healthy',
 18: 'Tomato___Bacterial_spot',
 19: 'Tomato___healthy'}

In [100]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32,3,input_shape=(224,224,3)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Conv2D(64,3,activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Conv2D(128,3,activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dense(20,activation="softmax"))

In [101]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['acc'])

In [103]:
history = model.fit(training_data,validation_data=testing_data,epochs=numEPOCHS,steps_per_epoch=100)

Epoch 1/3
100/100 [==============================] - 881s 9s/step - loss: 1.9247 - acc: 0.4928 - val_loss: 23.9794 - val_acc: 0.0458
Epoch 2/3
100/100 [==============================] - 679s 7s/step - loss: 1.6085 - acc: 0.5520 - val_loss: 24.0437 - val_acc: 0.1033
Epoch 3/3
100/100 [==============================] - 599s 6s/step - loss: 1.3711 - acc: 0.6136 - val_loss: 26.6688 - val_acc: 0.1439


In [2]:
model.save("crop_agent.h5")

In [8]:
from tensorflow import keras

import warnings
warnings.filterwarnings("ignore")

In [9]:
model = keras.models.load_model('crop_agent.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp1j1h8ow_/assets


INFO:tensorflow:Assets written to: /tmp/tmp1j1h8ow_/assets
2023-06-10 19:09:40.914449: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-06-10 19:09:40.914529: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-06-10 19:09:40.914973: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp1j1h8ow_
2023-06-10 19:09:40.925284: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-10 19:09:40.925353: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp1j1h8ow_
2023-06-10 19:09:40.951711: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-06-10 19:09:41.174319: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp1j1h8ow_
2023-06-10 19:09:41.208009: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

In [10]:
with open('cropAgent.tflite', 'wb') as f:
    f.write(tflite_model)

In [11]:
interpreter = tf.lite.Interpreter(model_path='cropAgent.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [15]:
image_path = 'cropdisease/valid/Apple___Cedar_apple_rust/025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655_newGRR.JPG'
image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224,3))
input_image = tf.keras.preprocessing.image.img_to_array(image)
input_image = tf.expand_dims(input_image, axis=0)
input_image /= 255.0  

In [16]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], input_image)
interpreter.invoke()
predictions = interpreter.get_tensor(output_details[0]['index'])

In [18]:
prediction_result = np.argmax(predictions)

In [19]:
prediction_result

5

In [32]:
prediction_result = reversed_dict[prediction_result]
prediction_result

'Corn_(maize)___Common_rust_'

In [35]:
prediction_result = re.sub(r'_{1,}', ' ', prediction_result)
print(prediction_result)

Corn (maize) Common rust 


## LLamaIndex Integration

In [44]:
!pip install -qU langchain openai tiktoken

In [41]:
import openai
import os
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass()

········


In [42]:
os.environ["WOLFRAM_ALPHA_APPID"] = getpass()

········


In [46]:
!pip install -qU wolframalpha
!pip install -qU wikipedia

In [43]:
about = f"Please tell me about {prediction_result} crop"

In [44]:
caused = f"How is {prediction_result} crop diseased caused?"

In [45]:
cure = f"What is cure for {prediction_result} crop diseased?"

In [47]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
#
llm = ChatOpenAI(temperature=0)
#define tools for the agent
tools = load_tools(['wikipedia','wolfram-alpha'],llm=llm)
#Setting a memory for conversation
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
#Define Agent
agent = initialize_agent(tools,llm,agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         verbose=True,
                         memory=memory,
                         max_iterations=6)

In [48]:
try:
    info = agent.run(input=about)
except Exception as e:
    info =str(e)
    if info.startswith("Could not parse LLM output: `"):
        info = info.removeprefix("Could not parse LLM output: `").removesuffix("`")
    else:
        raise Exception(str(e))



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Corn common rust
Observation: Page: Rust (fungus)
Summary: Rusts are plant diseases caused by pathogenic fungi of the order Pucciniales (previously known as Uredinales).
An estimated 168 rust genera and approximately 7,000 species, more than half of which belong to the genus Puccinia, are currently accepted. Rust fungi are highly specialized plant pathogens with several unique features. Taken as a group, rust fungi are diverse and affect many kinds of plants. However, each species has a very narrow range of hosts and cannot be transmitted to non-host plants. In addition, most rust fungi cannot be grown easily in pure culture.
A single species of rust fungi may be able to infect two different plant hosts in different stages of its life cycle, and may produce up to five morphologically and cytologically distinct spore-producing structures viz., spermogonia, aecia, uredinia, telia

In [49]:
print(info)

Do I need to use a tool? No 

Corn common rust is a plant disease caused by the fungus Puccinia sorghi. It infects corn and some species from the plant genus Oxalis. The disease is characterized by the appearance of small, circular, reddish-brown pustules on the leaves, stems, and ears of the corn plant. These pustules contain spores that can spread the disease to other plants. Common rust can reduce crop yields and quality, but can be managed through the use of resistant corn varieties and fungicides.


In [50]:
try:
    cause = agent.run(input=caused)
except Exception as e:
    cause =str(e)
    if cause.startswith("Could not parse LLM output: `"):
        cause = info.removeprefix("Could not parse LLM output: `").removesuffix("`")
    else:
        raise Exception(str(e))



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Corn common rust
Observation: Page: Rust (fungus)
Summary: Rusts are plant diseases caused by pathogenic fungi of the order Pucciniales (previously known as Uredinales).
An estimated 168 rust genera and approximately 7,000 species, more than half of which belong to the genus Puccinia, are currently accepted. Rust fungi are highly specialized plant pathogens with several unique features. Taken as a group, rust fungi are diverse and affect many kinds of plants. However, each species has a very narrow range of hosts and cannot be transmitted to non-host plants. In addition, most rust fungi cannot be grown easily in pure culture.
A single species of rust fungi may be able to infect two different plant hosts in different stages of its life cycle, and may produce up to five morphologically and cytologically distinct spore-producing structures viz., spermogonia, aecia, uredinia, telia

In [51]:
print(cause)

Do I need to use a tool? No 

Corn common rust is a plant disease caused by the fungus Puccinia sorghi. It infects corn and some species from the plant genus Oxalis. The disease is characterized by the appearance of small, circular, reddish-brown pustules on the leaves, stems, and ears of the corn plant. These pustules contain spores that can spread the disease to other plants. Common rust can reduce crop yields and quality, but can be managed through the use of resistant corn varieties and fungicides.


In [52]:
try:
    precaution = agent.run(input=cure)
except Exception as e:
    precaution =str(e)
    if precaution.startswith("Could not parse LLM output: `"):
        precaution = info.removeprefix("Could not parse LLM output: `").removesuffix("`")
    else:
        raise Exception(str(e))



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Wikipedia
Action Input: Cure for Corn Common Rust
Observation: Page: Linseed oil
Summary: Linseed oil, also known as flaxseed oil or flax oil (in its edible form), is a colourless to yellowish oil obtained from the dried, ripened seeds of the flax plant (Linum usitatissimum). The oil is obtained by pressing, sometimes followed by solvent extraction. 
Owing to its polymer-forming properties, linseed oil is often blended with combinations of other oils, resins or solvents as an impregnator, drying oil finish or varnish in wood finishing, as a pigment binder in oil paints, as a plasticizer and hardener in putty, and in the manufacture of linoleum. Linseed oil use has declined over the past several decades with increased availability of synthetic alkyd resins—which function similarly but resist yellowing.Linseed oil is an edible oil in demand as a dietary supplement, as a source of α-Linolenic acid, an om

In [53]:
print(precaution)

Unfortunately, there is no cure for corn common rust. However, there are some management practices that can help reduce the impact of the disease, such as planting resistant varieties, crop rotation, and timely application of fungicides. It is also important to monitor the crop regularly and remove any infected plants to prevent the spread of the disease.
